In [1]:
import csv
import os
import sys
from os.path import dirname, realpath,join
from PyQt5.QtGui import QIcon 
from PyQt5.QtWidgets import QApplication, QMainWindow,QVBoxLayout,QAction,QFileDialog, QWidget, QSlider, QMessageBox, QTableWidget, QTableWidgetItem, QSizePolicy
from PyQt5.uic import  loadUiType
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import Qt
from PyQt5.QtCore import QUrl
import json
from PyQt5.QtWebEngineWidgets import QWebEngineView
import subprocess
import pandas as pd
import re
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
%matplotlib inline
import numpy as np
from pandas.plotting import register_matplotlib_converters
from datetime import datetime
from statsmodels.tsa.seasonal import STL

import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import folium
from folium.plugins import HeatMap
from folium.plugins import Draw
from folium.plugins import MarkerCluster
from sklearn.cluster import KMeans

In [2]:
class MplCanvas(FigureCanvas):

    def __init__(self, parent=None, width=5, height=5, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)
        

class Ui_Healthcare(object):
    def setupUi(self, Healthcare):
        Healthcare.setObjectName("Healthcare")
        Healthcare.resize(1700, 917)
        self.centralwidget = QtWidgets.QWidget(Healthcare)
        self.centralwidget.setObjectName("centralwidget")
        
        self.groupBox = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox.setGeometry(QtCore.QRect(20, 10, 161, 261))
        self.groupBox.setStyleSheet("border-color: rgb(0, 0, 0);")
        self.groupBox.setObjectName("groupBox")
        self.groupBox_2 = QtWidgets.QGroupBox(self.groupBox)
        self.groupBox_2.setGeometry(QtCore.QRect(0, 180, 161, 71))
        self.groupBox_2.setObjectName("groupBox_2")
        
        self.Age_1 = QtWidgets.QSpinBox(self.groupBox_2)
        self.Age_1.setGeometry(QtCore.QRect(30, 30, 42, 22))
        self.Age_1.setObjectName("Age_1")
        
        #setting spin box properties
        
        self.Age_1.setMinimum(12)
        self.Age_1.setMaximum(80)
        
        self.Age_2 = QtWidgets.QSpinBox(self.groupBox_2)
        self.Age_2.setGeometry(QtCore.QRect(100, 30, 42, 22))
        self.Age_2.setObjectName("Age_2")
        
        #setting spin box properties
        
        self.Age_2.setMinimum(12)
        self.Age_2.setMaximum(80)
        
        self.label = QtWidgets.QLabel(self.groupBox_2)
        self.label.setGeometry(QtCore.QRect(80, 30, 21, 16))
        self.label.setObjectName("label")
        
        self.Age_Lim1 = QtWidgets.QCheckBox(self.groupBox_2, clicked = lambda: self.age_lim1_sel())
        self.Age_Lim1.setGeometry(QtCore.QRect(10, 30, 16, 20))
        self.Age_Lim1.setText("")
        self.Age_Lim1.setObjectName("Age_Lim1")
        
        self.All_Gen = QtWidgets.QRadioButton(self.groupBox, clicked = lambda: self.gender_selected())
        self.All_Gen.setGeometry(QtCore.QRect(30, 60, 95, 20))
        self.All_Gen.setObjectName("All_Gen")
        
        self.Male = QtWidgets.QRadioButton(self.groupBox, clicked = lambda: self.gender_selected())
        self.Male.setGeometry(QtCore.QRect(30, 90, 95, 20))
        self.Male.setObjectName("Male")
        
        self.Female = QtWidgets.QRadioButton(self.groupBox, clicked = lambda: self.gender_selected())
        self.Female.setGeometry(QtCore.QRect(30, 120, 95, 20))
        self.Female.setObjectName("Female")
        
        self.Undefined = QtWidgets.QRadioButton(self.groupBox)
        self.Undefined.setGeometry(QtCore.QRect(30, 150, 95, 20))
        self.Undefined.setObjectName("Undefined")
        
        self.TimeSeries = QtWidgets.QCheckBox(self.centralwidget, clicked = lambda: self.Value())
        self.TimeSeries.setGeometry(QtCore.QRect(30, 550, 121, 20))
        self.TimeSeries.setObjectName("TimeSeries")
        
        self.KDE = QtWidgets.QCheckBox(self.centralwidget, clicked = lambda: self.plot_kde())
        self.KDE.setGeometry(QtCore.QRect(30, 580, 111, 20))
        self.KDE.setObjectName("KDE")
        
        self.Anomalies = QtWidgets.QCheckBox(self.centralwidget, clicked = lambda: self.Value())
        self.Anomalies.setGeometry(QtCore.QRect(30, 670, 121, 20))
        self.Anomalies.setObjectName("Anomalies")
        
        self.Forecast = QtWidgets.QCheckBox(self.centralwidget, clicked = lambda: self.Value())
        self.Forecast.setGeometry(QtCore.QRect(30, 610, 121, 20))
        self.Forecast.setObjectName("Forecast")
        
        self.Days_forecast = QtWidgets.QSpinBox(self.centralwidget)
        self.Days_forecast.setGeometry(QtCore.QRect(50, 640, 51, 22))
        self.Days_forecast.setObjectName("Days_forecast")
        
        self.Aggregation = QtWidgets.QCheckBox(self.centralwidget, clicked = lambda: self.Data_Aggregation())
        self.Aggregation.setGeometry(QtCore.QRect(30, 700, 121, 20))
        self.Aggregation.setObjectName("Aggregation")
        
        #setting spin box properties
        
        self.Days_forecast.setMinimum(1)
        self.Days_forecast.setMaximum(30)
        
        self.line = QtWidgets.QFrame(self.centralwidget)
        self.line.setGeometry(QtCore.QRect(190, 0, 20, 801))
        self.line.setFrameShape(QtWidgets.QFrame.VLine)
        self.line.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.line.setObjectName("line")
        
        self.widget = QtWidgets.QWidget(self.centralwidget)
        self.widget.setGeometry(QtCore.QRect(210, 10, 591, 801))
        self.widget.setObjectName("widget")
        
        #making canvas for KDE
        self.layout = QtWidgets.QVBoxLayout(self.widget)
        self.webview = QWebEngineView()
        self.layout.addWidget(self.webview)
        
        # Load data
        lahore = gpd.read_file(r"Lahore towns uc and vill\LH_TOWNS-2006_GEO_AREAS.shp")
        dataset = pd.read_csv("ABCD.csv")
        dataset = gpd.GeoDataFrame(dataset, geometry=gpd.points_from_xy(dataset.Longitude, dataset.Latitude))

        # Clip data to Lahore boundary
        dataset = gpd.clip(dataset, lahore)

        # Create KDE plot
        kde = folium.plugins.HeatMap(dataset[['Latitude', 'Longitude']], 
                                     name='KDE', 
                                     radius=15, 
                                     blur=25)

        # Create map centered on Lahore
        m = folium.Map(location=[31.582045, 74.329376], zoom_start=12)

        # Add KDE plot to map
        kde.add_to(m)

        # Add Lahore regions shapefile to map
        folium.GeoJson(lahore).add_to(m)

        # Add layer control
        folium.LayerControl().add_to(m)

        # Add draw plugin to map
        draw = Draw(export=True)
        draw.add_to(m)
        
        # Save map as HTML file
        m.save('kde_map.html')
        
        # Load the HTML file generated by Folium
        self.html_path = os.path.abspath('kde_map.html')

        # Connect to the downloadRequested signal
        self.webview.page().profile().downloadRequested.connect(self.handle_download_requested)
        
        #ending canvas making procedure
        
        #////////////////for Data Aggregation on same canvas of KDE//////////////////////////////////////////////
        
        # Read in the patient data
        df = pd.read_csv('ABCD.csv')

        # Read in the shapefile
        shapefile = r"Lahore towns uc and vill\LH_TOWNS-2006_GEO_AREAS.shp"
        gdf = gpd.read_file(shapefile)
        
        kmeans = KMeans(n_clusters=5, random_state=0).fit(df[['Latitude', 'Longitude']])
        df['cluster'] = kmeans.labels_

        # Create a map centered on the mean latitude and longitude
        center_lat = np.mean(df['Latitude'])
        center_lon = np.mean(df['Longitude'])
        map = folium.Map(location=[center_lat, center_lon], zoom_start=10)

        # Create a MarkerCluster layer for each cluster
        for i in range(kmeans.n_clusters):
            cluster_df = df[df['cluster'] == i]
            marker_cluster = MarkerCluster()
            for index, row in cluster_df.iterrows():
                marker = folium.Marker(location=[row['Latitude'], row['Longitude']])
                marker_cluster.add_child(marker)
            map.add_child(marker_cluster)

        # Add the shapefile to the map
        folium.GeoJson(gdf).add_to(map)

        # Display the map and save it to an HTML file
        map.save('patient_map.html')
        
        self.widget_2 = QtWidgets.QWidget(self.centralwidget)
        self.widget_2.setGeometry(QtCore.QRect(800, 10, 411, 281))
        self.widget_2.setObjectName("widget_2")
        
        
        #making canvas for time series analysis
        self.canvas_1 = MplCanvas(self, width=5, height=5, dpi=100)
        self.layout_1 = QtWidgets.QVBoxLayout(self.widget_2)
        self.layout_1.addWidget(self.canvas_1)
        #ending canvas making procedure
        
        self.widget_3 = QtWidgets.QWidget(self.centralwidget)
        self.widget_3.setGeometry(QtCore.QRect(800, 290, 411, 281))
        self.widget_3.setObjectName("widget_3")
        
        #making canvas for Anomalies
        self.canvas_2 = MplCanvas(self, width=5, height=5, dpi=100)
        self.layout_2 = QtWidgets.QVBoxLayout(self.widget_3)
        self.layout_2.addWidget(self.canvas_2)
        #ending canvas making procedure
        
        self.widget_4 = QtWidgets.QWidget(self.centralwidget)
        self.widget_4.setGeometry(QtCore.QRect(800, 570, 411, 281))
        self.widget_4.setObjectName("widget_4")
        
        #making canvas for Anomalies
        self.canvas_3 = MplCanvas(self, width=5, height=5, dpi=100)
        self.layout_3 = QtWidgets.QVBoxLayout(self.widget_4)
        self.layout_3.addWidget(self.canvas_3)
        #ending canvas making procedure
        
        self.horizontalSlider = QtWidgets.QSlider(self.centralwidget)
        self.horizontalSlider.setGeometry(QtCore.QRect(200, 830, 591, 22))
        self.horizontalSlider.setOrientation(QtCore.Qt.Horizontal)
        self.horizontalSlider.setObjectName("horizontalSlider")
        
        #set slider properties
        self.horizontalSlider.setMinimum(7)
        self.horizontalSlider.setMaximum(123)
        self.horizontalSlider.setTickPosition(QSlider.TicksBelow)
        self.horizontalSlider.setTickInterval(7)
        self.horizontalSlider.setSingleStep(7)
        
        #move the slider
        self.horizontalSlider.valueChanged.connect(self.slide_it)
        
        self.pushButton = QtWidgets.QPushButton(self.centralwidget, clicked = lambda: self.age_lim1_sel())
        self.pushButton.setGeometry(QtCore.QRect(30, 730, 93, 28))
        self.pushButton.setObjectName("pushButton")
        
        self.groupBox_3 = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox_3.setGeometry(QtCore.QRect(20, 290, 161, 231))
        self.groupBox_3.setObjectName("groupBox_3")
        
        self.Search_1 = QtWidgets.QLineEdit(self.groupBox_3)
        self.Search_1.setGeometry(QtCore.QRect(30, 60, 113, 22))
        self.Search_1.setObjectName("Search_1")
        
        self.Search_2 = QtWidgets.QLineEdit(self.groupBox_3)
        self.Search_2.setGeometry(QtCore.QRect(30, 130, 113, 22))
        self.Search_2.setObjectName("Search_2")
        
        self.Search_3 = QtWidgets.QLineEdit(self.groupBox_3)
        self.Search_3.setGeometry(QtCore.QRect(30, 200, 113, 22))
        self.Search_3.setObjectName("Search_3")
        
        self.FirstComplaint = QtWidgets.QCheckBox(self.groupBox_3, clicked = lambda: self.complaint())
        self.FirstComplaint.setGeometry(QtCore.QRect(10, 30, 111, 20))
        self.FirstComplaint.setObjectName("FirstComplaint")
        
        self.SecondComplaint = QtWidgets.QCheckBox(self.groupBox_3, clicked = lambda: self.complaint())
        self.SecondComplaint.setGeometry(QtCore.QRect(10, 100, 111, 20))
        self.SecondComplaint.setObjectName("SecondComplaint")
        
        self.ThirdComplaint = QtWidgets.QCheckBox(self.groupBox_3, clicked = lambda: self.complaint())
        self.ThirdComplaint.setGeometry(QtCore.QRect(10, 170, 111, 20))
        self.ThirdComplaint.setObjectName("ThirdComplaint")
        
        self.tableWidget = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget.setGeometry(QtCore.QRect(1220, 10, 471, 841))
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setColumnCount(0)
        self.tableWidget.setRowCount(0)
        
        Healthcare.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(Healthcare)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 1116, 26))
        self.menubar.setObjectName("menubar")
        self.menuFile = QtWidgets.QMenu(self.menubar)
        self.menuFile.setObjectName("menuFile")
        self.menuEdit = QtWidgets.QMenu(self.menubar)
        self.menuEdit.setObjectName("menuEdit")
        self.menuHelp = QtWidgets.QMenu(self.menubar)
        self.menuHelp.setObjectName("menuHelp")
        Healthcare.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(Healthcare)
        self.statusbar.setObjectName("statusbar")
        Healthcare.setStatusBar(self.statusbar)
        self.actionImport = QtWidgets.QAction(Healthcare)
        self.actionImport.setObjectName("actionImport")
        self.actionOpen = QtWidgets.QAction(Healthcare)
        self.actionOpen.setObjectName("actionOpen")
        self.actionSave = QtWidgets.QAction(Healthcare)
        self.actionSave.setObjectName("actionSave")
        self.menuFile.addAction(self.actionImport)
        self.menuFile.addAction(self.actionOpen)
        self.menuFile.addAction(self.actionSave)
        self.menubar.addAction(self.menuFile.menuAction())
        self.menubar.addAction(self.menuEdit.menuAction())
        self.menubar.addAction(self.menuHelp.menuAction())

        self.retranslateUi(Healthcare)
        QtCore.QMetaObject.connectSlotsByName(Healthcare)
        
        #opening file explorer
        path = self.actionImport.triggered.connect(self.browsefiles)

    def retranslateUi(self, Healthcare):
        _translate = QtCore.QCoreApplication.translate
        Healthcare.setWindowTitle(_translate("Healthcare", "MainWindow"))
        self.All_Gen.setText(_translate("Healthcare", "All"))
        self.Male.setText(_translate("Healthcare", "Male"))
        self.Female.setText(_translate("Healthcare", "Female"))
        self.Undefined.setText(_translate("Healthcare", "Undefined "))
        self.groupBox.setTitle(_translate("Healthcare", "Demographics"))
        self.groupBox_2.setTitle(_translate("Healthcare", "Age"))
        self.label.setText(_translate("Healthcare", "To"))
        self.TimeSeries.setText(_translate("Healthcare", "Trend"))
        self.KDE.setText(_translate("Healthcare", "Kernel Density"))
        self.Forecast.setText(_translate("Healthcare", "Time Forecasting"))
        self.Anomalies.setText(_translate("Healthcare", "Anomalies"))
        self.Aggregation.setText(_translate("Healthcare", "Data Aggregation"))
        self.pushButton.setText(_translate("Healthcare", "Delete Current"))
        self.groupBox_3.setTitle(_translate("Healthcare", "Keyword Classifiers"))
        self.FirstComplaint.setText(_translate("Healthcare", "1st Complaint"))
        self.SecondComplaint.setText(_translate("Healthcare", "2nd Complaint"))
        self.ThirdComplaint.setText(_translate("Healthcare", "3rd Complaint"))
        self.menuFile.setTitle(_translate("Healthcare", "File"))
        self.menuEdit.setTitle(_translate("Healthcare", "Edit"))
        self.menuHelp.setTitle(_translate("Healthcare", "Help"))
        self.actionImport.setText(_translate("Healthcare", "Import"))
        self.actionImport.setStatusTip(_translate("Healthcare", "Import Dataset"))
        self.actionOpen.setText(_translate("Healthcare", "Open"))
        self.actionSave.setText(_translate("Healthcare", "Save"))
        
    #user-defined functions
        
    def browsefiles(self):
        path=QFileDialog.getOpenFileName(None, 'Open file', os.getenv('HOME'), "CSV(*.csv)") 
        print(path[0])     
        self.df = pd.read_csv(path[0])
        # Convert Date column to datetime object
        self.df['Date'] = pd.to_datetime(self.df['Date'], format='%m/%d/%Y')

        # Format Date column as string with same format
        self.df['Date'] = self.df['Date'].dt.strftime('%Y-%m-%d')

        # Convert Date column back to datetime object
        self.df['Date'] = pd.to_datetime(self.df['Date'], format='%Y-%m-%d')

        # Assuming 'Date' is the name of the datetime column
        self.df.set_index('Date', inplace=True)
        
        register_matplotlib_converters()
        return self.df
        
    def Value(self):
        self.value = self.horizontalSlider.value()
        
        if self.All_Gen.isChecked() == True or self.Male.isChecked() == True or self.Female.isChecked() == True:
            try:
                # Read the contents of the saved file and print them
                with open(self.save_path, 'r') as f:
                    data = json.load(f)
                # Get the coordinates of the geometry
                geometry_type = data['features'][0]['geometry']['type']
                if geometry_type == 'Polygon':
                    coordinates = data['features'][0]['geometry']['coordinates'][0]
                elif geometry_type == 'MultiPolygon':
                    coordinates = []
                    for polygon in data['features'][0]['geometry']['coordinates']:
                        coordinates.extend(polygon[0])
                else:
                    raise ValueError('Unsupported geometry type')

                # Initialize the minimum and maximum latitude and longitude values
                self.min_lat, self.max_lat = coordinates[0][1], coordinates[0][1]
                self.min_lon, self.max_lon = coordinates[0][0], coordinates[0][0]

                # Iterate over the coordinates and update the minimum and maximum values
                for coord in coordinates:
                    lon, lat = coord
                    if lat < self.min_lat:
                        self.min_lat = lat
                    elif lat > self.max_lat:
                        self.max_lat = lat
                    if lon < self.min_lon:
                        self.min_lon = lon
                    elif lon > self.max_lon:
                        self.max_lon = lon
                self.dataframe = self.dataframe[(self.dataframe['Longitude']>=self.min_lon) & (self.dataframe['Longitude']<=self.max_lon)]
                self.dataframe = self.dataframe[(self.dataframe['Latitude']>=self.min_lat) & (self.dataframe['Latitude']<=self.max_lat)]
                numRows = len(self.dataframe)
        
                self.tableWidget.setColumnCount(len(self.dataframe.columns))
                self.tableWidget.setRowCount(numRows)
                self.tableWidget.setHorizontalHeaderLabels(self.dataframe.columns)
        
                for i in range(numRows):
                    for j in range(len(self.dataframe.columns)):
                        self.tableWidget.setItem(i,j,QTableWidgetItem(str(self.dataframe.iat[i,j])))
                
                self.tableWidget.resizeColumnsToContents()
                
                self.Time_Series_Forecasting(self.dataframe)
                    
                self.Total = self.dataframe['Age'].resample('D').count() #day of week effect
                self.checked(self.Total,self.value)
            
                return self.Total
            
            except:
                if self.TimeSeries.isChecked() == True or self.Anomalies.isChecked() == True or self.Forecast.isChecked() == True:
                    msg = QMessageBox()
                    msg.setText('You have not selected any region on Kernel Density plot!')
                    msg.setIcon(QMessageBox.Critical)
                    
                    x = msg.exec_()       
    
    def slide_it(self):
        self.value = self.horizontalSlider.value()
        if self.All_Gen.isChecked() == True or self.Male.isChecked() == True or self.Female.isChecked() == True:
            self.Total = self.dataframe['Age'].resample('D').count() #day of week effect
            self.checked(self.Total,self.value)
        
            return self.Total
    
    def checked(self,Total,value):
        if self.TimeSeries.isChecked() == True and self.Anomalies.isChecked() == True:
            self.canvas_1.axes.cla()
            self.canvas_2.axes.cla()
            self.timeseries()
            self.Anomalies_cal()
            
        elif self.Anomalies.isChecked() == True:
            self.canvas_1.axes.cla()
            self.canvas_2.axes.cla()
            self.canvas_1.draw()
            self.Anomalies_cal()
            
        elif self.TimeSeries.isChecked() == True:
            self.canvas_1.axes.cla()
            self.canvas_2.axes.cla()
            self.canvas_2.draw()
            self.timeseries()
            
        else:
            self.canvas_1.axes.cla()
            self.canvas_2.axes.cla()
            self.canvas_1.draw()
            self.canvas_2.draw()
            
    #Gender selection
    
    def gender_selected(self):
        if self.Male.isChecked() == True:
            
            if self.Age_Lim1.isChecked() == True:
                self.dataframe_age = self.df[(self.df['Age'] >= self.Age_1.value()) & (self.df['Age'] <= self.Age_2.value())]
                self.dataframe = self.dataframe_age[self.dataframe_age['Sex'] == 2]
            
            elif self.Age_Lim1.isChecked() == False:    
                self.dataframe = self.df[self.df['Sex'] == 2]
            
            if self.TimeSeries.isChecked() == True or self.Anomalies.isChecked() == True or self.Forecast.isChecked() == True:
                self.Value()
                
            return self.dataframe
        
        if self.All_Gen.isChecked() == True:
            
            if self.Age_Lim1.isChecked() == True:
                self.dataframe_age = self.df[(self.df['Age'] >= self.Age_1.value()) & (self.df['Age'] <= self.Age_2.value())]
                self.dataframe = self.dataframe_age
            
            elif self.Age_Lim1.isChecked() == False:    
                self.dataframe = self.df
            
            if self.TimeSeries.isChecked() == True or self.Anomalies.isChecked() == True or self.Forecast.isChecked() == True:
                self.Value()

            return self.dataframe
        
        if self.Female.isChecked() == True:
            
            if self.Age_Lim1.isChecked() == True:
                self.dataframe_age = self.df[(self.df['Age'] >= self.Age_1.value()) & (self.df['Age'] <= self.Age_2.value())]
                self.dataframe = self.dataframe_age[self.dataframe_age['Sex'] == 1]
            
            elif self.Age_Lim1.isChecked() == False:  
                self.dataframe = self.df[self.df['Sex'] == 1]
            
            if self.TimeSeries.isChecked() == True or self.Anomalies.isChecked() == True or self.Forecast.isChecked() == True:
                self.Value()
            
            return self.dataframe
        
        
    #Age limit selection
        
    def age_lim1_sel(self):
        if self.Age_Lim1.isChecked() == False:
            self.gender_selected()
            
            
        if self.All_Gen.isChecked() == True or self.Male.isChecked() == True or self.Female.isChecked() == True:
            
            self.dataframe = self.dataframe[(self.dataframe['Age'] >= self.Age_1.value()) & (self.dataframe['Age'] <= self.Age_2.value())]
            self.gender_selected()
            return self.dataframe
        
        else:
        
            self.dataframe_age = self.df[(self.df['Age'] >= self.Age_1.value()) & (self.df['Age'] <= self.Age_2.value())]
            return self.dataframe_age
        
    def complaint(self):
        if self.FirstComplaint.isChecked() == False:
            self.gender_selected()
            
        elif self.FirstComplaint.isChecked() == True and self.SecondComplaint.isChecked() == True and self.ThirdComplaint.isChecked() == True:
            self.dataframe = self.dataframe[self.dataframe['3rd Complaint'].str.contains(self.Search_3.text(),flags=re.IGNORECASE)]
            
        elif self.FirstComplaint.isChecked() == True and self.SecondComplaint.isChecked() == True:
            self.dataframe = self.dataframe[self.dataframe['2nd Complaint'].str.contains(self.Search_2.text(),flags=re.IGNORECASE)]
            
        elif self.FirstComplaint.isChecked() == True:
            self.dataframe = self.dataframe[self.dataframe['1st complaint'].str.contains(self.Search_1.text(),flags=re.IGNORECASE)]
        
        if self.TimeSeries.isChecked() == True or self.Anomalies.isChecked() == True or self.Forecast.isChecked() == True:
                self.Value()
        
        return self.dataframe
        
            
        
    #Plotting functions 
    
    def handle_download_requested(self, item):
        # Get the suggested file name from the download item
        suggested_filename = item.suggestedFileName()
        if suggested_filename:
            # Open a file dialog to choose the save path
            self.save_path, _ = QFileDialog.getSaveFileName(
                None, 'Save File', suggested_filename
            )
            if self.save_path:
                # Set the save path for the download item
                item.setPath(self.save_path)
                item.accept()
    
    def plot_kde(self):
        
        if self.KDE.isChecked() == False:
            # Clear the canvas by reloading a blank page
            self.webview.load(QUrl('about:blank'))
            return
        
        # Reload the Folium-generated HTML file
        self.webview.load(QUrl.fromLocalFile(self.html_path))
        
    def Data_Aggregation(self):
        
        if self.Aggregation.isChecked() == False:
            # Clear the canvas by reloading a blank page
            self.webview.load(QUrl('about:blank'))
            return
        
        # Reload the Folium-generated HTML file
        self.html_path_1 = os.path.abspath('patient_map.html')
        self.webview.load(QUrl.fromLocalFile(self.html_path_1))
        
        
    def timeseries(self):
        if self.TimeSeries.isChecked() == False:
            self.canvas_1.axes.cla()
            self.canvas_1.draw()
            return
        
        else:
            result = STL(self.Total).fit()
            seasonal, trend, resid = result.seasonal, result.trend, result.resid
            
            trend = trend.iloc[self.value-7:self.value]

            self.canvas_1.axes.plot(trend)
            for tick in self.canvas_1.axes.get_xticklabels():
                tick.set_fontsize(6)
                
            for tick in self.canvas_1.axes.get_yticklabels():
                tick.set_fontsize(6)
            
            for tick in self.canvas_1.axes.get_xticklabels():
                tick.set_rotation(-10)
            
            self.canvas_1.axes.set_xlabel('Date', fontsize=4)
            self.canvas_1.axes.set_ylabel('No. of Patients')
            self.canvas_1.axes.set_title('Trend')
            
            self.canvas_1.draw()
        
    def Anomalies_cal(self):
        
        if self.Anomalies.isChecked() == False:
            self.canvas_2.axes.cla()
            self.canvas_2.draw()
            return
        
        else:
            result = STL(self.Total).fit()
            seasonal, trend, resid = result.seasonal, result.trend, result.resid
        
            estimated = trend + seasonal
            resid_mu = resid.mean()
            resid_dev = resid.std()

            lower = resid_mu - 2*resid_dev
            upper = resid_mu + 2*resid_dev
            
            self.Total = self.Total.iloc[self.value-7:self.value]
        
            anomalies = self.Total[(resid < lower) | (resid > upper)]
            self.canvas_2.axes.plot(self.Total)
    
            self.canvas_2.axes.scatter(anomalies.index, anomalies, color='r', marker='D')
        
            for tick in self.canvas_2.axes.get_xticklabels():
                tick.set_fontsize(6)
                
            for tick in self.canvas_2.axes.get_yticklabels():
                tick.set_fontsize(6)
            
            for tick in self.canvas_2.axes.get_xticklabels():
                tick.set_rotation(-10)
            
            self.canvas_2.axes.set_xlabel('Date', fontsize=4)
            self.canvas_2.axes.set_ylabel('No. of Patients')
            self.canvas_2.axes.set_title('Anomalies in Time Series')
        
            self.canvas_2.draw()
        
    def Time_Series_Forecasting(self,dataframe):
        if self.Forecast.isChecked() == False:
            self.canvas_3.axes.cla()
            self.canvas_3.draw()
            return
        
        else:
            self.canvas_3.axes.cla()
            self.days = self.dataframe.groupby(self.dataframe.index)[['Age']].count()
            self.days.columns = ['Frequency']
            
            from keras.models import load_model
            model = load_model('model.h5')
            x = self.days.shape
            
            X = {'F1': [self.days.iloc[x[0]-1]['Frequency']] , 'F2': [self.days.iloc[x[0]-2]['Frequency']] , 'F3': self.days.iloc[x[0]-3]['Frequency']}
            X = pd.DataFrame(X)
            Y = model.predict(X, verbose = 0)
            predicted = pd.DataFrame(Y, columns=['Predicted'])
            
            X = {'F1': [predicted.iloc[0]['Predicted']] , 'F2': [self.days.iloc[x[0]-1]['Frequency']] , 'F3': self.days.iloc[x[0]-2]['Frequency']}
            X = pd.DataFrame(X)
            Y = model.predict(X, verbose = 0)
            newpredicted = pd.DataFrame(Y, columns=['Predicted'])
            predicted = pd.concat([predicted, newpredicted])
            
            X = {'F1': [predicted.iloc[1]['Predicted']] , 'F2': [predicted.iloc[0]['Predicted']] , 'F3': self.days.iloc[x[0]-1]['Frequency']}
            X = pd.DataFrame(X)
            Y = model.predict(X, verbose = 0)
            newpredicted = pd.DataFrame(Y, columns=['Predicted'])
            predicted = pd.concat([predicted, newpredicted])
            
            for i in range(27):
                X = {'F1': [predicted.iloc[i+2]['Predicted']] , 'F2': [predicted.iloc[i+1]['Predicted']] , 'F3': [predicted.iloc[i]['Predicted']]}
                X = pd.DataFrame(X)
                Y = model.predict(X, verbose = 0)
                newpredicted = pd.DataFrame(Y, columns=['Predicted'])
                predicted = pd.concat([predicted, newpredicted])
                
            predicted = predicted.reset_index()
            predicted = predicted.drop(['index'],axis= 1)
            
            predicted = predicted.iloc[ : self.Days_forecast.value()]
            
            self.canvas_3.axes.plot(predicted, 'r--' ,label= 'Prediction Trend')
            
            for tick in self.canvas_3.axes.get_xticklabels():
                tick.set_fontsize(8)
                
            for tick in self.canvas_3.axes.get_yticklabels():
                tick.set_fontsize(6)
            
            self.canvas_3.axes.set_xlabel('days', fontsize=10)
            self.canvas_3.axes.set_ylabel('No. of Patients')
            self.canvas_3.axes.set_title('Results of prediction')
            
            self.canvas_3.draw()

In [3]:
if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    Healthcare = QtWidgets.QMainWindow()
    ui = Ui_Healthcare()
    ui.setupUi(Healthcare)
    Healthcare.show()
    sys.exit(app.exec_())

C:\Users\drfak\AppData\Local\Temp\ipykernel_3864\1792453793.py:117: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: GEOGCS["Geographic Coordinate System",DATUM["WGS84 ...

  dataset = gpd.clip(dataset, lahore)


SystemExit: 0

C:\Users\drfak\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
